## Compresses and Send Resting State Eyetracking Files to Project(.sh)

The script can be run immediately after [this](Auto_DAY1_Transfer.ipynb) one in your scratch folder. It will find the newly transferred files, sort them into their own folders, and copy a compressed version to their respective locations at projects.

*Remember to change the actual pathways*

In [ ]:
#!/bin/bash

destination_root="/home/samhsu03/projects/def-rolsen/AtRisk-Dataset/Resting_State_Eyetracking"

declare -a unsorted_dirs=()
declare -a created_participant_dirs=()
declare -a tarred_dirs=()

create_and_move_tar() {
  local source_dir=$1
  local cap_dir_name=$2
  local tar_name="${source_dir}.tar.gz"

  tar -czf "$tar_name" -C "$cap_dir_name" "$source_dir" && echo "Tarring complete for $source_dir"

  local dest_dir="$destination_root/$cap_dir_name"
  if [[ ! -d "$dest_dir" ]]; then
    mkdir -p "$dest_dir" && echo "Created directory at $dest_dir"
    created_participant_dirs+=("$dest_dir")
  fi

  mv "$tar_name" "$dest_dir" && echo "Moved $tar_name to $dest_dir"
  tarred_dirs+=("$source_dir")
}

for dir in */; do
  dir=${dir%/}
  if [[ "$dir" =~ [a-z] ]]; then
    unsorted_dirs+=("$dir")
    cap_dir_name=${dir^^}

    if [[ ! -d "$cap_dir_name" ]]; then
      mkdir "$cap_dir_name" && echo "Created Participant ID folder $cap_dir_name"
      created_participant_dirs+=("$cap_dir_name")
    fi

    mv "$dir" "$cap_dir_name/" && echo "Moved $dir to $cap_dir_name"
    create_and_move_tar "$dir" "$cap_dir_name"
  fi
done

echo "Summary:"
echo "Unsorted directories found:"
for dir in "${unsorted_dirs[@]}"; do
  echo "- $dir"
done
echo "Participant ID folders created at the source:"
for dir in "${created_participant_dirs[@]}"; do
  echo "- $dir"
done
echo "Unsorted directories tarred and moved:"
for dir in "${tarred_dirs[@]}"; do
  echo "- $dir moved to $destination_root/${dir^^}"
done
